## 3. Análise Exploratória com Dados Limpos  
📒 `2.1-rc-eda-cleaned.ipynb`

**Objetivo:** Explorar os dados tratados por meio de estatísticas descritivas e visualizações gráficas para identificar padrões, tendências e possíveis relações entre variáveis.

📊 **Atividades:**
- Relação entre Variáveis Númericas (scatterplots, heatmap)
- Relação entre Variáveis Categóricas e o Alvo (`price`)
- Insights Relevantes
- Next Steps


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from _utils import classificar_correlacao

%matplotlib inline

# Setup para mostrar todas as colunas do dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Desabilita mensagens de FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
file_path = "../data/processed/car_price_cleaned.csv"
df = pd.read_csv(file_path)

df.head()

#### Variáveis Numéricas

**Insights esperados**:
- Quais são as minhas variáveis númericas que existe correlação significativa com o a variável alvo `price`? Qual a força dessa correlação? Qual a relação linear?

In [77]:
# Deletar as colunas de `city-mpg` e `highway-mpg`,  em virtude da variável derivada que foi criada de consumo médio `avg-fuel-consumption`
# Deletar `symboling` variável categórica classificada por 'risk-classification'  
df.drop(['city-mpg', 'highway-mpg', 'symboling'], axis=1, inplace=True)

In [ ]:
# Estatistica Descritiva
df.describe()

In [78]:
# Seleciona as variáveis numéricas
features_numeric = df.select_dtypes(include=['int64', 'float64']).columns.to_list()

In [ ]:
# Matriz de correlação
plt.figure(figsize=(12, 8))
sns.heatmap(df[features_numeric].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação entre Variáveis Numéricas')
plt.savefig('../reports/figures/eda_heatmap_correlation')



**STEP 01:** Selecionar as variáveis preditoras que possuem correlação com a variável alvo `price`. Aqui iremos verificar se existe correlação e qual a a sua força e se a relação é linear positiva ou negativa.

**NOTA:** Considerar o nível de confiança alpha=0.05 (para um intervalo de 95% de confiança)

- i) **p-value(p):** verifica se existe ou não correlação dentro de um intervalo de confiança de 95%
    - P-value > 0.05    Ho = ***não há diferança ou correlação significativa***
    - P-value < 0.05    Ha = ***há diferença ou correlação significativa***
- ii) **pearson_correlation(r):** medir a força da correlação, sendo que (+) a correlação é positiva, (-) a correlação é negativa.

| Valor de r (+ ou -) | Interpretação           |
|---------------------|-------------------------|
| 0.00 a 0.19         | correlação muito fraca  |
| 0.20 a 0.39         | correlação fraca        |
| 0.40 a 0.69         | correlação moderada     |
| 0.70 a 0.89         | correlação forte        |
| 0.90 a 1.00         | correlação muito forte  |

Fonte: SHIMAKURA, Silvia Emiko. Interpretação do coeficiente de correlação. LEG, UFPR,. 2006.

In [ ]:
# Histograma para distribuição (em matriz de subplots)
n = len(features_numeric)
cols = 3  # número de colunas da matriz de gráficos
rows = (n + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols*5, rows*3))
axes = axes.flatten()

for i, col in enumerate(features_numeric):
    sns.histplot(df[col], kde=True, bins=30, ax=axes[i])
    axes[i].set_title(f'Distribuição - {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequência')

# Remove eixos extras se houver
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
fig.savefig('../reports/figures/eda_hist_numerical')
plt.show()

#### Variáveis Categóricas


In [73]:
# Contagem de categorias (em matriz de subplots)
categorical_features = df.select_dtypes(include=['object']).columns.to_list()
n = len(categorical_features)
cols = 3
rows = (n + cols - 1) // cols

In [ ]:
fig, axes = plt.subplots(rows, cols, figsize=(cols*5, rows*4))
axes = axes.flatten()

for i, col in enumerate(categorical_features):
    ax=axes[i]
    counts = df[col].value_counts()
    sns.countplot(data=df, x=col, order=df[col].value_counts().index, ax=axes[i] )
    axes[i].set_title(f'Contagem - {col}')
    axes[i].tick_params(axis='x', rotation=45)

        # Adiciona labels nas barras
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='bottom', fontsize=10, color='black', xytext=(0, 3), textcoords='offset points')

for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
fig.savefig('../reports/figures/eda_hist_categorical')
plt.show()

In [ ]:
# Boxplot entre variáveis categóricas e alvo (price)
if 'price' in df.columns:
    n = len(categorical_features)
    cols = 2
    rows = (n + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(cols*6, rows*4))
    axes = axes.flatten()

    for i, col in enumerate(categorical_features):
        sns.boxplot(data=df, x=col, y='price', order=df[col].value_counts().index, ax=axes[i])
        axes[i].set_title(f'Preço por Categoria - {col}')
        axes[i].tick_params(axis='x', rotation=45)

    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()